In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import ttest_ind
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize
from scipy.stats import boxcox
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.datasets import load_files

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from statsmodels.tools.eval_measures import mse, rmse
from wordcloud import WordCloud
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.svm import SVC
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn import neighbors
from IPython.display import Image

from sklearn.preprocessing import normalize

import pydotplus
from sklearn import ensemble

import warnings

%matplotlib inline
sns.set()

warnings.filterwarnings('ignore')
import time
import cv2
import glob

Using TensorFlow backend.


In [2]:
app_events_df = (pd.read_csv('Data/talkingdata/app_events.csv'))

app_labels_df = (pd.read_csv('Data/talkingdata/app_labels.csv'))

events_df = (pd.read_csv('Data/talkingdata/events.csv'))

gender_age_test_df = (pd.read_csv('Data/talkingdata/gender_age_test.csv'))

gender_age_train_df = (pd.read_csv('Data/talkingdata/gender_age_train.csv'))

label_categories_df = (pd.read_csv('Data/talkingdata/label_categories.csv'))

phone_brand_device_model_df = (pd.read_csv('Data/talkingdata/phone_brand_device_model.csv'))

sample_submission_df = (pd.read_csv('Data/talkingdata/sample_submission.csv'))

In [3]:
df_app_events = app_events_df
df_app_labels = app_labels_df
df_events = events_df
df_gender_age_test = gender_age_test_df
df_gender_age_train = gender_age_train_df
df_label_categories = label_categories_df
df_phone_brand_device_model = phone_brand_device_model_df

## Data Exploration

In [7]:
print(
#    app_events_df.head(),
    app_events_df.info(),
#    app_events_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32473067 entries, 0 to 32473066
Data columns (total 4 columns):
event_id        int64
app_id          int64
is_installed    int64
is_active       int64
dtypes: int64(4)
memory usage: 991.0 MB
None


In [8]:
#app_events_df['event_id'].value_counts()

#NTS: turn event_id and app_id into top 20 or so bar graphs (or histograms?) (line graph?)

In [9]:
print(
    app_labels_df.head(10),
    app_labels_df.info(),
#    app_labels_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459943 entries, 0 to 459942
Data columns (total 2 columns):
app_id      459943 non-null int64
label_id    459943 non-null int64
dtypes: int64(2)
memory usage: 7.0 MB
                app_id  label_id
0  7324884708820027918       251
1 -4494216993218550286       251
2  6058196446775239644       406
3  6058196446775239644       407
4  8694625920731541625       406
5  8694625920731541625       407
6  1977658975649789753       406
7  1977658975649789753       407
8  7311663864768030840       256
9  5902120154267999338       256 None


In [10]:
#app_labels_df['label_id'].value_counts()

In [11]:
print(
#    events_df.head(),
    events_df.info(),
#    events_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 5 columns):
event_id     int64
device_id    int64
timestamp    object
longitude    float64
latitude     float64
dtypes: float64(2), int64(2), object(1)
memory usage: 124.1+ MB
None


In [12]:
#events_df['event_id'].value_counts()

In [13]:
print(
#    gender_age_train_df.head(),
    gender_age_train_df.info(),
#    gender_age_train_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
device_id    74645 non-null int64
gender       74645 non-null object
age          74645 non-null int64
group        74645 non-null object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB
None


In [14]:
#gender_age_train_df['device_id'].value_counts()

In [15]:
print(
#    gender_age_test_df.head(),
    gender_age_test_df.info(),
#    gender_age_test_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112071 entries, 0 to 112070
Data columns (total 1 columns):
device_id    112071 non-null int64
dtypes: int64(1)
memory usage: 875.6 KB
None


In [16]:
print(
#    label_categories_df.head(),
    label_categories_df.info(),
#    label_categories_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
label_id    930 non-null int64
category    927 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB
None


In [17]:
print(
#    phone_brand_device_model_df.head(),
    phone_brand_device_model_df.info(),
#    phone_brand_device_model_df.describe()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187245 entries, 0 to 187244
Data columns (total 3 columns):
device_id       187245 non-null int64
phone_brand     187245 non-null object
device_model    187245 non-null object
dtypes: int64(1), object(2)
memory usage: 4.3+ MB
None


In [4]:
df_phone_brand_device_model['encoded_model'] = None

In [5]:
df_phone_brand_device_model['encoded_model'] = df_phone_brand_device_model['device_model'].apply(lambda x: x.encode('unicode-escape'))

In [6]:
#.encode('unicode-escape')
#.encode(encoding="utf-8", errors="strict")

In [7]:
del(app_events_df, app_labels_df, events_df, gender_age_test_df, gender_age_train_df,
   label_categories_df, phone_brand_device_model_df)

## Feature Engineering

merging

In [8]:
start_time = time.time()

apps_labeled = pd.merge(df_app_labels,
                       df_app_events[['app_id', 'is_active']],
                       on='app_id').dropna().drop_duplicates()

print("--- %s seconds ---" % (time.time() - start_time))

--- 91.01103711128235 seconds ---


In [ ]:
start_time = time.time()

device_apps = pd.merge(df_events,
                       df_app_events[['app_id', 'is_active', 'event_id']],
                       on='event_id').dropna().drop_duplicates()

print("--- %s seconds ---" % (time.time() - start_time))

--- 38.64483714103699 seconds ---


In [ ]:
start_time = time.time()

ga_apps = pd.merge(df_gender_age_train,
                  device_apps[['app_id', 'is_active', 'device_id']],
                  on='device_id').dropna().drop_duplicates()

print("--- %s seconds ---" % (time.time() - start_time))

--- 11.294361114501953 seconds ---


In [ ]:
start_time = time.time()

ga_apps_brands = pd.merge(ga_apps,
                         df_phone_brand_device_model[['device_id', 'encoded_model']],
                         on='device_id').dropna().drop_duplicates()


print("--- %s seconds ---" % (time.time() - start_time))

--- 1.5375468730926514 seconds ---


In [ ]:
start_time = time.time()

ga_apps_labeled_brands = pd.merge(ga_apps_brands,
                                 apps_labeled[['app_id', 'label_id']],
                                 on='app_id').dropna().drop_duplicates()



print("--- %s seconds ---" % (time.time() - start_time))

--- 17.529872179031372 seconds ---


In [ ]:
del(df_app_events, df_app_labels, df_events, df_gender_age_test, df_gender_age_train, df_label_categories,
   df_phone_brand_device_model)

In [ ]:
# Convert gender into a binary output
ga_apps_labeled_brands['gender'] = ga_apps_labeled_brands['gender'].apply(lambda x: 1 if x == 'M' else 0 )

In [ ]:
ga_apps_labeled_brands.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7034350 entries, 0 to 13874875
Data columns (total 8 columns):
device_id        int64
gender           int64
age              int64
group            object
app_id           int64
is_active        int64
encoded_model    object
label_id         int64
dtypes: int64(6), object(2)
memory usage: 483.0+ MB


In [ ]:
#make dummy variables from the object variables
start_time = time.time()

ga_apps_labeled_brands_dums = pd.concat([ga_apps_labeled_brands, pd.get_dummies(
    ga_apps_labeled_brands[['encoded_model']], drop_first=True)], axis=1)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#check the correlation of various features with the target

np.abs(ga_apps_labeled_brands[
    ga_apps_labeled_brands.select_dtypes([
        'int64', 'float64']).columns].iloc[:,1:].corr().loc[:,'gender']).sort_values(ascending=False).head(35)

In [ ]:
sns.heatmap(ga_apps_labeled_brands.select_dtypes(['int64', 'float64']).corr())

In [ ]:
X = pd.concat([ga_apps_labeled_brands[['device_id', 'app_id', 'is_active', 'label_id']], ga_apps_labeled_brands_dums.select_dtypes(['uint8'])], axis=1, sort=False)
             
y = ga_apps_labeled_brands['gender']


In [ ]:
X_norm = normalize(X)

In [ ]:
n_comps = np.arange(50)
param_grid_pca = [{'pca__n_components':n_comps}]
pipe_tree_pca = make_pipeline(PCA())

In [ ]:
gs_pca = GridSearchCV(pipe_tree_pca, param_grid=param_grid_pca, cv=10)

In [ ]:
gs_pca.fit(X_norm,y)
print(gs_pca.best_params_)

In [ ]:
#using the results of GridSearchCV to perform PCA

sklearn_pca = PCA(n_components=300)  
X_pca = sklearn_pca.fit_transform(X_norm)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA:\n',
    sklearn_pca.explained_variance_ratio_
)

#X_pca is now our PCA engineered feature set.

In [ ]:
# Split the data into four equally-sized samples for analysis with silhouette. 
X_half1, X_half2, X_pcahalf1, X_pcahalf2 = train_test_split(
    X_norm,
    X_pca,
    test_size=0.5,
    random_state=42)
X1, X2, X_pca1, X_pca2 = train_test_split(
    X_half1,
    X_pcahalf1,
    test_size=0.5,
    random_state=42)
X3, X4, X_pca3, X_pca4 = train_test_split(
    X_half2,
    X_pcahalf2,
    test_size=0.5,
    random_state=42)

In [ ]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_pca,
    y,
    test_size=0.1,
    random_state=42)

In [ ]:
del(ga_apps_labeled_brands, ga_apps_labeled_brands_dums, ga_apps_brands, ga_apps, device_apps, apps_labeled
   )

## Model 1: Unsupervised Clustering

In [ ]:
for sample in [X_pca1, X_pca2, X_pca3, X_pca4]:
    model = KMeans(n_clusters=4, random_state=42).fit(sample)
    labels_test = model.labels_
    print(metrics.silhouette_score(sample, labels_test, metric='euclidean'))

In [ ]:
for sample in [X_pca1, X_pca2, X_pca3, X_pca4]:
    model = KMeans(n_clusters=5, random_state=42).fit(sample)
    labels_test = model.labels_
    print(metrics.silhouette_score(sample, labels_test, metric='euclidean'))

In [ ]:
for sample in [X_pca1, X_pca2, X_pca3, X_pca4]:
    model = KMeans(n_clusters=6, random_state=42).fit(sample)
    labels_test = model.labels_
    print(metrics.silhouette_score(sample, labels_test, metric='euclidean'))

#### Silhouette analysis seems relatively ambivalent to the number of clusters from 4-6. We will try six clusters for our model.

In [ ]:
y_pred = KMeans(n_clusters=6).fit(X_train)
labels = y_pred.labels_

In [ ]:
print('Comparing k-means clusters against the data:')
print(pd.crosstab(y_train, labels))

## Model 2: Decision Tree

In [33]:
#setting up parameters for GridSearchCV(DecisionTreeClassifier())

depths = np.arange(1, 25)
num_features = [1, 2]

param_grid_dt_pca = [{'decisiontreeclassifier__max_depth':depths,
               'decisiontreeclassifier__max_features':num_features}]

pipe_tree_dt_pca = make_pipeline(tree.DecisionTreeClassifier(criterion='entropy'))

In [34]:
gs_dt_pca = GridSearchCV(pipe_tree_dt_pca, param_grid=param_grid_dt_pca, cv=10)

In [35]:
gs_dt_pca.fit(X_train,y_train)
print(gs_dt_pca.best_params_)

{'decisiontreeclassifier__max_depth': 1, 'decisiontreeclassifier__max_features': 1}


In [ ]:
#initialize and train the decision tree model using GridSearchCV results

decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=2,
    max_depth=1
)

decision_tree.fit(X_train, y_train)

In [ ]:
#making predictions for model evaluation
y_pred_dt = decision_tree.fit(X_train, y_train).predict(X_test)

In [ ]:
#evaluating the model performance through xvalidation of the training set

start_time = time.time()

print(cross_val_score(decision_tree, X_train, y_train, cv=10))
print('The 10-fold cross validation average for the training set is ', 
      cross_val_score(decision_tree, X_train, y_train, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#evaluating the model performance through xvalidation of the testing set

print(cross_val_score(decision_tree, X_test, y_test, cv=10))
print('The 10-fold cross validation average for the testing set  is ', 
      cross_val_score(decision_tree, X_test, y_test, cv=10).mean())

In [ ]:
#classification report

print(classification_report(y_test, y_pred_dt, labels=None, target_names=None, 
                      sample_weight=None, digits=2, output_dict=False
                     ))

In [ ]:
#confusion matrix
print('Test set confusion matrix:', '\n', 
      confusion_matrix(y_test, y_pred_dt, labels=None, sample_weight=None)
     )

In the classification reports, a value of 1 corresponds to a value of "late" for the target variable "delay", and 0 corresponds to a "on-time."  Although decision tree models tend to handle varied types of data well, this model isn't achieving any useful discrimination of the target. 

## Model 3: Random Forest Classifier

In [ ]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(X_train,y_train)

In [ ]:
#making predictions for model evaluation
y_preds_rfc=rfc.predict(X_test)

In [ ]:
#evaluating the model performance through xvalidation of the training set

start_time = time.time()

print(cross_val_score(rfc, X_train, y_train, cv=10))
print('The 10-fold cross validation average for the training set is ', 
      cross_val_score(rfc, X_train, y_train, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#evaluating the model performance through xvalidation of the testing set

print(cross_val_score(rfc, X_test, y_test, cv=10))
print('The 10-fold cross validation average for the testing set  is ', 
      cross_val_score(rfc, X_test, y_test, cv=10).mean())

In [ ]:
#classification report

print(classification_report(y_test, y_preds_rfc, labels=None, target_names=None, 
                      sample_weight=None, digits=2, output_dict=False
                     ))

In [ ]:
#confusion matrix
print('Test set confusion matrix:', '\n', 
      confusion_matrix(y_test, y_preds_rfc, labels=None, sample_weight=None)
     )

Similarly to the decision tree model, the random forest model is not a good fit for this data.  It was unable to discriminate the target variable whatsoever with this dataset. 

## Model 4: SVM

In [24]:
#kernels = ['linear', 'poly', 'rbf', 'sigmoid']
degrees = [1,2]
param_grid_svm = [{'svc__degree':degrees}]
pipe_tree_svm = make_pipeline(SVC(kernel = 'poly'))

In [25]:
gs_svm = GridSearchCV(pipe_tree_svm, param_grid=param_grid_svm, cv=10)

#gs_svm.get_params().keys()

In [39]:
gs_svm.fit(X_train,y_train)
print(gs_svm.best_params_)

{'svc__degree': 1}


In [25]:
svm = SVC(kernel='poly',degree=1)
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=1, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [26]:
#Setting the predictions for analysis
y_pred_svm = svm.fit(X_train, y_train).predict(X_test)

#evaluating the model performance through xvalidation of the training set

start_time = time.time()

print(cross_val_score(svm, X_train, y_train, cv=10))
print('The 10-fold cross validation average for the training set is ', 
      cross_val_score(svm, X_train, y_train, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

In [27]:
#evaluating the model performance through xvalidation of the testing set

start_time = time.time()

print(cross_val_score(svm, X_test, y_test, cv=10))
print('The 10-fold cross validation average for the testing set  is ', 
      cross_val_score(svm, X_test, y_test, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

[0.51049815 0.51049815 0.51049815 0.51029654 0.51029654 0.51029654
 0.51029654 0.5105068  0.5105068  0.5105068 ]
The 10-fold cross validation average for the testing set  is  0.510420099915708
--- 204.898530960083 seconds ---


In [28]:
#classification report

print(classification_report(y_test, y_pred_svm, labels=None, target_names=None, 
                      sample_weight=None, digits=2, output_dict=False
                     ))

              precision    recall  f1-score   support

           0       0.51      1.00      0.68     12393
           1       0.00      0.00      0.00     11887

    accuracy                           0.51     24280
   macro avg       0.26      0.50      0.34     24280
weighted avg       0.26      0.51      0.34     24280



In [29]:
#confusion matrix
print('Test set confusion matrix:', '\n', 
      confusion_matrix(y_test, y_pred_svm, labels=None, sample_weight=None)
     )

Test set confusion matrix: 
 [[12393     0]
 [11887     0]]


Similarly to the decision tree and random forest models above, the SVM model was completely unable to classify the target variable.

## Model 5: Naive Bayes

In [ ]:
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

# Fit our model to the data.
bnb.fit(X_train, y_train)

# Classify, storing the result in a new variable.
y_preds_nb = bnb.predict(X_test)

In [ ]:
#evaluating the model performance through xvalidation of the training set

start_time = time.time()

print(cross_val_score(bnb, X_train, y_train, cv=10))
print('The 10-fold cross validation average for the training set is ', 
      cross_val_score(bnb, X_train, y_train, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#evaluating the model performance through xvalidation of the testing set

print(cross_val_score(bnb, X_test, y_test, cv=10))
print('The 10-fold cross validation average for the testing set  is ', 
      cross_val_score(bnb, X_test, y_test, cv=10).mean())

In [ ]:
#classification report

print(classification_report(y_test, y_preds_nb, labels=None, target_names=None, 
                      sample_weight=None, digits=2, output_dict=False
                     ))

In [ ]:
#confusion matrix
print('Test set confusion matrix:', '\n', 
      confusion_matrix(y_test, y_preds_nb, labels=None, sample_weight=None)
     )

## Model 6: KNN

In [40]:
#kernels = ['linear', 'poly', 'rbf', 'sigmoid']
num_neighbors = np.arange(1, 25)
weights = ['distance','uniform']
leaf_sizes = np.arange(1, 40)

param_grid_knn = [{'kneighborsclassifier__n_neighbors':num_neighbors,
                      'kneighborsclassifier__weights':weights,
                      'kneighborsclassifier__leaf_size':leaf_sizes}
                     ]
pipe_tree_knn = make_pipeline(neighbors.KNeighborsClassifier())

In [41]:
gs_knn = GridSearchCV(pipe_tree_knn, param_grid=param_grid_knn, cv=10)

#gs_knn.get_params().keys()

In [42]:
gs_knn.fit(X_train,y_train)
print(gs_knn.best_params_)

{'kneighborsclassifier__leaf_size': 4, 'kneighborsclassifier__n_neighbors': 21, 'kneighborsclassifier__weights': 'uniform'}


In [ ]:
#{'kneighborsclassifier__leaf_size': 29, 'kneighborsclassifier__n_neighbors': 1, 'kneighborsclassifier__weights': 'distance'}
knn = neighbors.KNeighborsClassifier(n_neighbors=21, weights='distance', leaf_size=4)
knn.fit(X_train, y_train)

In [ ]:
# Classify, storing the result in a new variable.
y_preds_knn = knn.predict(X_test)

In [ ]:
#evaluating the model performance through xvalidation of the training set

start_time = time.time()

print(cross_val_score(knn, X_train, y_train, cv=10))
print('The 10-fold cross validation average for the training set is ', 
      cross_val_score(knn, X_train, y_train, cv=10).mean())

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#evaluating the model performance through xvalidation of the testing set

print(cross_val_score(knn, X_test, y_test, cv=10))
print('The 10-fold cross validation average for the testing set  is ', 
      cross_val_score(knn, X_test, y_test, cv=10).mean())

In [ ]:
#classification report

print(classification_report(y_test, y_preds_knn, labels=None, target_names=None, 
                      sample_weight=None, digits=2, output_dict=False
                     ))

In [ ]:
#confusion matrix
print('Test set confusion matrix:', '\n', 
      confusion_matrix(y_test, y_preds_knn, labels=None, sample_weight=None)
     )

## Conclusion